# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [335]:
# your code here
# Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
import pandas as pd
tic_tac_toe = pd.read_csv("tic-tac-toe.csv")

display(tic_tac_toe.head())
display(tic_tac_toe.info())

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


None

In [336]:
# Convert the categorical values to numeric in all columns.
tic_tac_toe.replace({"x": 1, "o": -1, "b": 0}, inplace=True)

tic_tac_toe['class'].replace({True: 1, False: 0}, inplace=True)
display(tic_tac_toe.head())


/var/folders/67/kwhhh9lj76v79c9zbkp29ms40000gn/T/ipykernel_64535/548841578.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tic_tac_toe.replace({"x": 1, "o": -1, "b": 0}, inplace=True)
/var/folders/67/kwhhh9lj76v79c9zbkp29ms40000gn/T/ipykernel_64535/548841578.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the origin

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,-1,-1,1,-1,-1,1
1,1,1,1,1,-1,-1,-1,1,-1,1
2,1,1,1,1,-1,-1,-1,-1,1,1
3,1,1,1,1,-1,-1,-1,0,0,1
4,1,1,1,1,-1,-1,0,-1,0,1


In [337]:
# Separate the inputs and output.
output = tic_tac_toe['class']
output = pd.DataFrame(output)


inputs = tic_tac_toe.drop(columns=['class'])

In [338]:
# Normalize the input data.
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

inputs_scaled = scaler.fit_transform(inputs)
inputs_scaled = pd.DataFrame(inputs_scaled, columns=inputs.columns)
inputs_scaled.columns
# Question i have: shoul i not normalize only after train and test split??

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'], dtype='object')

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [339]:
# your code here
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(inputs_scaled,output,test_size=0.2)

# Check shapes after splitting
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (766, 9), y_train shape: (766, 1)
X_test shape: (192, 9), y_test shape: (192, 1)


In [340]:


# Model Architecture
model = Sequential()
model.add(Dense(units=16,activation = 'relu', input_shape=(9,))) # input_shape is the number of features
model.add(Dense(units=32,activation = 'relu'))
model.add(Dense(units=64,activation = 'relu'))
model.add(Dense(units=2,activation = 'softmax')) # unit= 2 = nr. of classes | softmax for probability distr

# Compile the model : define parameters for the training of the model (optimizer, loss function, perf metrics)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )

# Train the model
model.fit(X_train,y_train, epochs = 3)

# Make predictions of test data
y_pred = model.predict(X_test)


# Get the predicted class by choosing the index with the highest probability\
y_pred = np.argmax(y_pred, axis=1)



Epoch 1/3


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.4762 - loss: 0.7074
Epoch 2/3
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.6749 - loss: 0.6001
Epoch 3/3
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - accuracy: 0.7076 - loss: 0.5845
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step


In [341]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - accuracy: 0.7625 - loss: 0.5266
Test Loss: 0.5397854447364807
Test Accuracy: 0.7447916865348816


In [342]:
# Save the model

# Save the model in TensorFlow SavedModel format
model.save('my_first_deep_neural_net.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [343]:
display(X_test.iloc[0])
display(y_test.iloc[0])
display(X_test.iloc[3])
display(y_test.iloc[3])
display(X_test.iloc[10])
display(y_test.iloc[10])

y_pred_0 = model.predict(X_test.iloc[0].values.reshape(1, -1))
y_pred_3 = model.predict(X_test.iloc[3].values.reshape(1, -1))
y_pred_10 = model.predict(X_test.iloc[10].values.reshape(1, -1))

print("prediction on row 1 of X_test:", y_pred_0)
print("prediction on row 2 of X_test:", y_pred_3)
print("prediction on row 9 of X_test:", y_pred_10)

# Only 1/3 preictions are correct

TL    1.0
TM    0.5
TR    0.0
ML    1.0
MM    1.0
MR    1.0
BL    0.0
BM    0.5
BR    0.0
Name: 240, dtype: float64

class    1
Name: 240, dtype: int64

TL    1.0
TM    1.0
TR    1.0
ML    0.0
MM    0.0
MR    0.5
BL    0.0
BM    0.5
BR    1.0
Name: 30, dtype: float64

class    1
Name: 30, dtype: int64

TL    0.0
TM    1.0
TR    0.0
ML    1.0
MM    1.0
MR    0.5
BL    0.5
BM    1.0
BR    0.0
Name: 327, dtype: float64

class    1
Name: 327, dtype: int64

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
prediction on row 1 of X_test: [[0.3356835 0.6643165]]
prediction on row 2 of X_test: [[0.374257 0.625743]]
prediction on row 9 of X_test: [[0.33095998 0.66904   ]]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [344]:

# Model Architecture
model = Sequential()
model.add(Dense(units=16,activation = 'relu', input_shape=(9,))) # input_shape is the number of features
model.add(Dense(units=32,activation = 'relu'))
model.add(Dense(units=64,activation = 'relu'))
model.add(Dense(units=2,activation = 'softmax')) # unit= 2 = nr. of classes | softmax for probability distr

from keras.optimizers import Adam
# Compile the model : define parameters for the training of the model (optimizer, loss function, perf metrics)
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )

# Train the model
model.fit(X_train,y_train, epochs = 10)

# Make predictions of test data
y_pred = model.predict(X_test)


# Get the predicted class by choosing the index with the highest probability\
y_pred = np.argmax(y_pred, axis=1)


# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - accuracy: 0.6454 - loss: 0.6341
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - accuracy: 0.7436 - loss: 0.5097
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - accuracy: 0.8053 - loss: 0.4377
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.8576 - loss: 0.3198
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.8984 - loss: 0.2397
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.9310 - loss: 0.1986
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - accuracy: 0.9599 - loss: 0.1377
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.9693 - loss: 0.0895
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - accuracy: 0.9663 - loss: 0.0886
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.9474 - loss: 0.1742
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.9609 - loss: 0.0884
Test Loss: 0.10210319608449936
Test Ac

**Which approach(es) did you find helpful to improve your model performance?**

In [345]:
# your answer here
#adjusting the nr. of epochs
